<a href="https://colab.research.google.com/github/ArsT1982/Spark_DZ2/blob/main/Spark_DZ_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install --quiet pyspark

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Practice").getOrCreate()

In [3]:
from pyspark import SparkFiles

covid_data_file_url = "https://raw.githubusercontent.com/glincow/netology-spark-sql/main/data/covid-data.csv"
spark.sparkContext.addFile(covid_data_file_url)
file_path  = 'file://' + SparkFiles.get('covid-data.csv')
df = spark.read.option('inferSchema', 'true').option('header', 'true').csv(file_path)

In [18]:
df.show()

+--------+---------+-----------+----------+-----------+---------+------------------+------------+----------+-------------------+-----------------------+---------------------+------------------------------+------------------------+----------------------+-------------------------------+-----------------+------------+------------------------+-------------+-------------------------+---------------------+---------------------------------+----------------------+----------------------------------+---------+-----------+------------------------+----------------------+------------------+-------------------------------+-------------+--------------+-----------+------------------+-----------------+-----------------------+----------------+-------------------------+------------------------------+-----------------------------+-----------------------------------+-------------------------------------+----------------+-----------+------------------+----------+-------------+-------------+--------------+--

In [95]:
Find_OWID_from_df = df.filter(F.col('iso_code').like('OWID_%'))
Exclude_OWID_from_df = df.subtract(Find_OWID_from_df)
#Exclude_OWID_from_df.show()

df_null_to_zero=Exclude_OWID_from_df.na.fill(0)
#df_null_to_zero.show()


+--------+-------------+-------------------+----------+-----------+---------+------------------+------------+----------+-------------------+-----------------------+---------------------+------------------------------+------------------------+----------------------+-------------------------------+-----------------+------------+------------------------+-------------+-------------------------+---------------------+---------------------------------+----------------------+----------------------------------+---------+-----------+------------------------+----------------------+------------------+-------------------------------+-------------+--------------+---------------+------------------+-----------------+-----------------------+----------------+-------------------------+------------------------------+-----------------------------+-----------------------------------+-------------------------------------+----------------+-----------+------------------+----------+-------------+-------------+-

In [106]:
from pyspark.sql import functions as F

df_perc_recov_by_cntry = df_null_to_zero.select('iso_code', 'location',F.round(((F.col('total_cases')-F.col('total_deaths'))*100.0/F.col('population')),2).alias('recovered,%')).filter(df_null_to_zero.date == '2020-03-31').sort(F.col('recovered,%').desc())
df_perc_recov_by_cntry.show(15)

+--------+-------------+-----------+
|iso_code|     location|recovered,%|
+--------+-------------+-----------+
|     VAT|      Vatican|       0.74|
|     SMR|   San Marino|       0.62|
|     AND|      Andorra|       0.47|
|     LUX|   Luxembourg|       0.34|
|     ISL|      Iceland|       0.33|
|     CHE|  Switzerland|       0.19|
|     ESP|        Spain|       0.19|
|     LIE|Liechtenstein|       0.18|
|     ITA|        Italy|       0.15|
|     MCO|       Monaco|       0.13|
|     AUT|      Austria|       0.11|
|     BEL|      Belgium|        0.1|
|     DEU|      Germany|       0.08|
|     NOR|       Norway|       0.08|
|     FRA|       France|       0.07|
+--------+-------------+-----------+
only showing top 15 rows



In [108]:
from pyspark.sql import functions as F

df_new_cases_week = df_null_to_zero.select('iso_code', 'location', 'date', 'new_cases').filter(df_null_to_zero.date.between ('2021-03-22','2021-03-28')).groupBy("location").agg(F.max("new_cases").alias("max new cases")).sort(F.col('max new cases').desc())
df_new_cases_week.show(10)

+-------------+-------------+
|     location|max new cases|
+-------------+-------------+
|       Brazil|     100158.0|
|United States|      86960.0|
|        India|      68020.0|
|       France|      65392.0|
|       Poland|      35145.0|
|       Turkey|      30021.0|
|        Italy|      24501.0|
|      Germany|      23757.0|
|         Peru|      19206.0|
|      Ukraine|      18226.0|
+-------------+-------------+
only showing top 10 rows



In [109]:
from pyspark.sql import functions as F

df_new_cases_week = df_null_to_zero.select('iso_code', 'location', 'date', 'new_cases').filter(df_null_to_zero.date.between ('2021-03-22','2021-03-28')).groupBy("location").agg(F.sum("new_cases").alias("max new cases")).sort(F.col('max new cases').desc())
df_new_cases_week.show(10)

+-------------+-------------+
|     location|max new cases|
+-------------+-------------+
|       Brazil|     536455.0|
|United States|     442739.0|
|        India|     393563.0|
|       France|     263088.0|
|       Turkey|     195051.0|
|       Poland|     192441.0|
|        Italy|     155681.0|
|      Germany|     114651.0|
|      Ukraine|      98367.0|
|         Peru|      63556.0|
+-------------+-------------+
only showing top 10 rows



In [114]:
from pyspark.sql import functions as F

df_new_cases_week = df_null_to_zero.select('iso_code', 'location', 'date', 'new_cases').filter(df_null_to_zero.date.between ('2021-03-22','2021-03-28')).filter(F.col('location').like('Russia')).sort(F.col('date').asc())
df_new_cases_week.show(10)

+--------+--------+----------+---------+
|iso_code|location|      date|new_cases|
+--------+--------+----------+---------+
|     RUS|  Russia|2021-03-22|   9195.0|
|     RUS|  Russia|2021-03-23|   8369.0|
|     RUS|  Russia|2021-03-24|   8769.0|
|     RUS|  Russia|2021-03-25|   9128.0|
|     RUS|  Russia|2021-03-26|   9073.0|
|     RUS|  Russia|2021-03-27|   8783.0|
|     RUS|  Russia|2021-03-28|   8979.0|
+--------+--------+----------+---------+



In [125]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql.functions import lag


#df_new_cases_week = df_null_to_zero.select('iso_code', 'location', 'date', 'new_cases').filter(df_null_to_zero.date.between ('2021-03-22','2021-03-28')).filter(F.col('location').like('Russia')).sort(F.col('date').asc())

w = Window.orderBy("date")
leadDf = df_null_to_zero.select('iso_code', 'location', 'date', 'new_cases').filter(df_null_to_zero.date.between ('2021-03-22','2021-03-28')).filter(F.col('location').like('Russia')).sort(F.col('date').asc())

df_itog = leadDf.withColumn("yesterday_cases", lag('new_cases', 1, 0).over(w)).show()

#df_new_cases_week.show(10)

#df_new_cases_week.show(10)





+--------+--------+----------+---------+-------+
|iso_code|location|      date|new_cases|new_col|
+--------+--------+----------+---------+-------+
|     RUS|  Russia|2021-03-22|   9195.0|    0.0|
|     RUS|  Russia|2021-03-23|   8369.0| 9195.0|
|     RUS|  Russia|2021-03-24|   8769.0| 8369.0|
|     RUS|  Russia|2021-03-25|   9128.0| 8769.0|
|     RUS|  Russia|2021-03-26|   9073.0| 9128.0|
|     RUS|  Russia|2021-03-27|   8783.0| 9073.0|
|     RUS|  Russia|2021-03-28|   8979.0| 8783.0|
+--------+--------+----------+---------+-------+

